## Kidney-related laboratory data after receiving the second dose of medication

In [ ]:
import root_config as rc

rc.configure()

from detectdd.auth_bigquery import BigQueryClient
from detectdd.serializer import Serializer

try:
    cohort = Serializer().read_cohort()  # need to run 01-cohort.ipynb to produce the cohort
except FileNotFoundError:
    raise Exception("Need to run [01-cohort.ipynb] at least once to create the cohort file in the /out directory")


big_query = BigQueryClient.auth()

cohort.describe()

In [ ]:
icu = 'physionet-data.mimiciv_icu'

In [ ]:
def fetch_itemids():
    query_template = """
    SELECT itemid, label
    FROM `physionet-data.mimiciv_hosp.d_labitems`
    WHERE LOWER(label) LIKE '%{keyword}%'
    """
    keywords = ["creatinine", "bun", "uric acid"]
    item_ids = {}

    for keyword in keywords:
        query = query_template.format(keyword=keyword)
        result = big_query.query(query).result().to_dataframe()
        item_ids[keyword] = result['itemid'].tolist()

    return item_ids

item_ids = fetch_itemids()
item_ids



In [ ]:
# 'creatinine': [52024,] 'bun': [51842],'uric acid': [51007]

In [ ]:
def chartevents_query():
    query = f"""
    SELECT subject_id, hadm_id, itemid, charttime, valuenum, valueuom
    FROM `physionet-data.mimiciv_hosp.labevents`
    WHERE itemid = 52024
    LIMIT 10
    """
    result = big_query.query(query).result().to_dataframe()
    return result

sample_records = chartevents_query()
print(sample_records)


In [ ]:
from detectdd.query_multiplexer import QueryMultiplexer, WhereClauseGenerator
from detectdd import config

def fetch_kidney_function_data():
    item_ids = {
        'creatinine': [52024],
        'bun': [51842],
        'uric acid': [51007]
    }
    
    sql_template = f"""
    SELECT le.subject_id AS subject_id,
            le.hadm_id AS hadm_id,
            le.itemid AS item_id,
            le.charttime AS chart_time,
            le.valuenum AS lab_value,
            le.valueuom AS unit
    FROM `physionet-data.mimiciv_hosp.labevents` AS le
    WHERE le.itemid IN {tuple(item_ids['creatinine'] + item_ids['bun'] + item_ids['uric acid'])}
    AND ($where)
    """
    
    where_fragment = "(le.subject_id= $subject_id AND DATETIME_DIFF(le.charttime, '$dose_b_time', HOUR) BETWEEN 0 AND 12)"

    multimap_data = {k: v.tolist() for k, v in cohort.groupby('subject_id')['dose_b_time']}
    
    query_multiplexer = QueryMultiplexer(BigQueryClient().auth())

    results = query_multiplexer.multiplex_query(sql_template, multi_map_data=multimap_data, where_clause=WhereClauseGenerator(where_fragment, "subject_id", "dose_b_time"))
    
    return results

kidney_function_results = fetch_kidney_function_data()
kidney_function_results
